In [1]:
%%bash 
date 

ls -la 

pwd

python.exe --version 

ollama.exe list 


Fri Mar 21 15:13:15 EDT 2025
total 36
drwxrwxrwx 1 jmake jmake   512 Mar 21 15:13 .
drwxrwxrwx 1 jmake jmake   512 Mar 20 20:38 ..
drwxrwxrwx 1 jmake jmake   512 Mar 21 13:59 .ipynb_checkpoints
-rwxrwxrwx 1 jmake jmake 11716 Mar 21 13:56 04_agentic_rag_1a1.ipynb
-rwxrwxrwx 1 jmake jmake 18740 Mar 21 15:13 04_agentic_rag_1b1.ipynb
drwxrwxrwx 1 jmake jmake   512 Mar 20 20:56 ChromaDB
drwxrwxrwx 1 jmake jmake   512 Mar 20 20:45 Data
-rwxrwxrwx 1 jmake jmake     4 Mar 20 20:57 app_4.py
-rwxrwxrwx 1 jmake jmake  2187 Mar 20 20:40 ingest_pdfs.py
/mnt/d/z2025_1/AI/RAGs/Test4
Python 3.12.6


In [2]:
%%time 
## Created '6709' chunks from 'Data/Blender43Manual.pdf' at './ChromaDB'
#!python.exe ingest_pdfs.py 

CPU times: total: 0 ns
Wall time: 0 ns


In [3]:
%%time 
## python.exe -m pip install langchain_ollama
import smolagents
from langchain_ollama.llms import OllamaLLM

hf_token = ""
model_id = "Qwen/Qwen2.5-Coder-32B-Instruct"

engine = smolagents.HfApiModel(model_id=model_id, token=hf_token, max_tokens=5000)
print( engine.to_dict() )

messages = [{"role": "user", "content": [{"type": "text", "text": "Hello, how are you?"}]}]

response = engine(messages, stop_sequences=["END"])
print(response.content)


D:\z2025_1\AI\RAGs\Env4\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'max_tokens': 5000, 'last_input_token_count': None, 'last_output_token_count': None, 'model_id': 'Qwen/Qwen2.5-Coder-32B-Instruct', 'provider': None}
Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?
CPU times: total: 1.19 s
Wall time: 1.56 s


In [4]:
%%time
import smolagents

# Create the reasoner for better RAG
reasoner = smolagents.CodeAgent(tools=[], model=engine, add_base_tools=False, max_steps=2)
print( list(reasoner.to_dict().keys()) )


This agent has step_callbacks: they will be ignored by this method. 1

['tools', 'model', 'managed_agents', 'prompt_templates', 'max_steps', 'verbosity_level', 'grammar', 'planning_interval', 'name', 'description', 'requirements', 'authorized_imports', 'executor_type', 'executor_kwargs', 'max_print_outputs_length']
CPU times: total: 46.9 ms
Wall time: 27.5 ms


In [5]:
%%time 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

def split_documents_into_chunks( documents ) : 
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
    chunks = text_splitter.split_documents(documents)
    return chunks


def load_and_process_pdfs(data_dir: str) :
    loader = DirectoryLoader(data_dir, glob="**/*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return split_documents_into_chunks(documents)


data_dir = "Data"
pdf_chunks = load_and_process_pdfs(data_dir)
print( len(pdf_chunks) ) # 6709 

6709
CPU times: total: 31.8 s
Wall time: 32.1 s


In [6]:
%%time 
from langchain_community.document_loaders import RecursiveUrlLoader

def load_and_process_html(url) :
    loader = RecursiveUrlLoader(url)
    docs = loader.load()
    ## metadata = docs[0].metadata
    ## page_content = docs[0].page_content[:]
    return split_documents_into_chunks(docs)

url = "https://docs.python.org/3.9/"
html_chunks = load_and_process_html(url) 
print( len(html_chunks) ) # 2102 

D:\z2025_1\AI\RAGs\Env4\Lib\site-packages\langchain_community\document_loaders\recursive_url_loader.py:43: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(raw_html, "html.parser")


2102
CPU times: total: 1.06 s
Wall time: 2.63 s


In [7]:
## pdf_agent

In [8]:
## python.exe -m pip install rank_bm25
import smolagents
from langchain_community.retrievers import BM25Retriever

class RetrieverTool(smolagents.Tool):
    name = "retriever"
    description = "Uses semantic search to retrieve the parts of transformers documentation that could be most relevant to answer your query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        self.retriever = BM25Retriever.from_documents(
            docs, k=10
        )

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.retriever.invoke(
            query,
        )
        return "\nRetrieved documents:\n" + "".join(
            [
                f"\n\n===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

In [9]:
%%time
import smolagents

pdf_retriever_tool = RetrieverTool(pdf_chunks)
print( pdf_retriever_tool )

pdf_agent = smolagents.CodeAgent(tools=[pdf_retriever_tool], model=engine, max_steps=2, verbosity_level=0)
print( pdf_agent )

CPU times: total: 234 ms
Wall time: 228 ms


In [10]:
%%time 
query = "List the Principled BSDF Inputs"  
pdf_agent_output = pdf_agent.run(query)
print(f"Final output: '{pdf_agent_output}' ") ## 22

Reached max steps.

Final output: 'Certainly! Here is the list of Principled BSDF Inputs based on the provided documents:

1. Base Color
2. Subsurface
3. Subsurface Radius
4. Subsurface Color
5. Metallic
6. Specular
7. Specular Tint
8. Roughness
9. Anisotropic
10. Anisotropic Rotation
11. Sheen
12. Sheen Tint
13. Clearcoat
14. Clearcoat Roughness
15. IOR (Index of Refraction)
16. Transmission
17. Transmission Roughness
18. Absorption Color
19. Absorption Distance
20. Scatter
21. Scatter Radius
22. Scatter Anisotropy
23. Normal
24. Clearcoat Normal
25. Tangent

These inputs are commonly used in rendering and computer graphics software, particularly in the Cycles render engine used by Blender.' 
CPU times: total: 15.6 ms
Wall time: 24.9 s


In [11]:
%%time 
query = "Waht is the meaning of BSDF?"
pdf_agent_output = pdf_agent.run(query)
print(f"Final output: '{pdf_agent_output}' ")  ## Bidirectional Scattering Distribution Function

Final output: 'Bidirectional Scattering Distribution Function' 
CPU times: total: 15.6 ms
Wall time: 9.26 s


In [12]:
## html_agent

In [13]:
%%time
import smolagents

html_retriever_tool = RetrieverTool(html_chunks)
print( html_retriever_tool )

html_agent = smolagents.CodeAgent(tools=[html_retriever_tool], model=engine, max_steps=2, verbosity_level=0)
print( html_agent )

query = "Recommended third party tools for 'Extending and Embedding the Python Interpreter' mentioned in these documents"  
html_agent_output = html_agent.run(query)
print(f"Final output: '{html_agent_output}' ") ## 4,  Cython, cffi, SWIG and Numba

Final output: '['Cython', 'cffi', 'SWIG', 'Numba']' 
CPU times: total: 2.23 s
Wall time: 12.6 s


In [14]:
%%bash 
## jupyter-nbconvert.exe --to notebook --execute .\agentic_rag_1a1.ipynb --output executed_notebook.ipynb
date 

Fri Mar 21 15:14:39 EDT 2025
